# K-Means clustering using scikit-learn

In this notebook we apply scikit-learn’s K-Means implementation to the normalized zone-hour feature matrix in order to segment demand patterns into behavioral clusters. The resulting cluster labels are used in the following tasks for evaluation, visualization and business interpretation.

In [2]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

DATA_PATH = "../../data/processed/clustering_features_scaled.parquet"

In [3]:
X_scaled_df = pd.read_parquet(DATA_PATH)
X_scaled_df.shape

(296807, 6)

In [4]:
X = X_scaled_df.values

In [5]:
X

array([[-0.51060548,  2.80433372,  2.57368633, -0.2688393 , -1.03270344,
        -0.63153158],
       [-0.51771742,  7.69128887,  3.09159913,  0.03688014, -1.03270344,
        -0.63153158],
       [-0.51771742,  5.76963871,  7.83481719,  0.49545931, -1.03270344,
        -0.63153158],
       ...,
       [-0.51771742,  1.18285857,  2.09677496,  1.41261765,  1.5004962 ,
         1.58345209],
       [-0.51060548,  0.02026022,  0.3240026 ,  1.56547738,  1.5004962 ,
         1.58345209],
       [-0.51771742, -0.62481786, -0.60068754,  1.7183371 ,  1.5004962 ,
         1.58345209]], shape=(296807, 6))

### Fit K-means
We start with a fixed number of clusters and evaluate the results in later steps.

In [6]:
k = 4
kmeans = KMeans(
    n_clusters=k,
    random_state=42,
    n_init=10
)

kmeans.fit(X)

,"n_clusters n_clusters: int, default=8The number of clusters to form as well as the number ofcentroids to generate.For an example of how to choose an optimal value for `n_clusters` refer to:ref:`sphx_glr_auto_examples_cluster_plot_kmeans_silhouette_analysis.py`.",4
,"init init: {'k-means++', 'random'}, callable or array-like of shape (n_clusters, n_features), default='k-means++'Method for initialization:* 'k-means++' : selects initial cluster centroids using sampling based on an empirical probability distribution of the points' contribution to the overall inertia. This technique speeds up convergence. The algorithm implemented is ""greedy k-means++"". It differs from the vanilla k-means++ by making several trials at each sampling step and choosing the best centroid among them.* 'random': choose `n_clusters` observations (rows) at random from data for the initial centroids.* If an array is passed, it should be of shape (n_clusters, n_features) and gives the initial centers.* If a callable is passed, it should take arguments X, n_clusters and a random state and return an initialization.For an example of how to use the different `init` strategies, see:ref:`sphx_glr_auto_examples_cluster_plot_kmeans_digits.py`.For an evaluation of the impact of initialization, see the example:ref:`sphx_glr_auto_examples_cluster_plot_kmeans_stability_low_dim_dense.py`.",'k-means++'
,"n_init n_init: 'auto' or int, default='auto'Number of times the k-means algorithm is run with different centroidseeds. The final results is the best output of `n_init` consecutive runsin terms of inertia. Several runs are recommended for sparsehigh-dimensional problems (see :ref:`kmeans_sparse_high_dim`).When `n_init='auto'`, the number of runs depends on the value of init:10 if using `init='random'` or `init` is a callable;1 if using `init='k-means++'` or `init` is an array-like... versionadded:: 1.2 Added 'auto' option for `n_init`... versionchanged:: 1.4 Default value for `n_init` changed to `'auto'`.",10
,"max_iter max_iter: int, default=300Maximum number of iterations of the k-means algorithm for asingle run.",300
,"tol tol: float, default=1e-4Relative tolerance with regards to Frobenius norm of the differencein the cluster centers of two consecutive iterations to declareconvergence.",0.0001
,"verbose verbose: int, default=0Verbosity mode.",0
,"random_state random_state: int, RandomState instance or None, default=NoneDetermines random number generation for centroid initialization. Usean int to make the randomness deterministic.See :term:`Glossary `.",42
,"copy_x copy_x: bool, default=TrueWhen pre-computing distances it is more numerically accurate to centerthe data first. If copy_x is True (default), then the original data isnot modified. If False, the original data is modified, and put backbefore the function returns, but small numerical differences may beintroduced by subtracting and then adding the data mean. Note that ifthe original data is not C-contiguous, a copy will be made even ifcopy_x is False. If the original data is sparse, but not in CSR format,a copy will be made even if copy_x is False.",True
,"algorithm algorithm: {""lloyd"", ""elkan""}, default=""lloyd""K-means algorithm to use. The classical EM-style algorithm is `""lloyd""`.The `""elkan""` variation can be more efficient on some datasets withwell-defined clusters, by using the triangle inequality. However it'smore memory intensive due to the allocation of an extra array of shape`(n_samples, n_clusters)`... versionchanged:: 0.18 Added Elkan algorithm.. versionchanged:: 1.1 Renamed ""full"" to ""lloyd"", and deprecated ""auto"" and ""full"". Changed ""auto"" to use ""lloyd"" instead of ""elkan"".",'lloyd'


Extract cluster assignments and centroid coordinates.

In [7]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [8]:
labels.shape

(296807,)

In [9]:
centroids.shape

(4, 6)

In [10]:
centroids

array([[ 2.38819189, -0.49701459, -0.55443855,  0.5512178 , -0.25125029,
        -0.38943952],
       [-0.29867274, -0.29207285, -0.30158807, -0.08789578, -0.5188236 ,
        -0.63153158],
       [-0.12707455, -0.27911753, -0.2717378 , -0.05733254,  1.25341023,
         1.58345209],
       [-0.41368863,  1.93672901,  2.00019919,  0.02453241, -0.16809482,
        -0.24768131]])

In [11]:
# Attach cluster labels to the normalized feature table
df_clusters = X_scaled_df.copy()
df_clusters["cluster"] = labels

df_clusters.head()

,demand,avg_fare,avg_distance,hour,day_of_week,is_weekend,cluster
0,-0.510605,2.804334,2.573686,-0.268839,-1.032703,-0.631532,3
1,-0.517717,7.691289,3.091599,0.036880,-1.032703,-0.631532,3
2,-0.517717,5.769639,7.834817,0.495459,-1.032703,-0.631532,3
3,-0.517717,0.733590,-0.799221,-1.491717,-0.526064,-0.631532,1
4,-0.517717,-0.260367,1.656549,-1.338857,-0.526064,-0.631532,1


Check the size of each cluster to ensure no empty or degenerate clusters exist.

In [12]:
np.bincount(labels)

array([ 30027, 151352,  74219,  41209])

In [13]:
AGG_PATH = "../../data/processed/nyc_demand_zone_hour_2019_q1.parquet"

# Load clean snapshot
df = pd.read_parquet(AGG_PATH)

In [14]:
df["cluster"] = labels

In [15]:
df.head()

,zone_id,pickup_hour_ts,demand,avg_fare,avg_distance,hour,day_of_week,is_weekend,day,month,cluster
0,1,2019-01-01 10:00:00,2,61.25,16.90,10,1,0,1,1,3
1,1,2019-01-01 12:00:00,1,135.00,19.30,12,1,0,1,1,3
2,1,2019-01-01 15:00:00,1,106.00,41.28,15,1,0,1,1,3
3,1,2019-01-02 02:00:00,1,30.00,1.27,2,2,0,2,1,1
4,1,2019-01-02 03:00:00,1,15.00,12.65,3,2,0,2,1,1


In [16]:
df.to_parquet(
    "../../data/processed/zone_hour_clusters.parquet"
)